# Parameter Tuning
    Find the best approach for training: Rolling Nested CV or not

In [1]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LinearRegression

# empty list to append metric values
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

# Data set processing (fixed test set (2 months)):

In [2]:
# data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)
data = data.loc[data.index > 2018110000, :]
    
# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)
    
# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

X.fillna(X.mean(), inplace = True)
y.fillna(y.mean(), inplace = True)
    
X = X.astype('float64')
X = X.round(20)
    
# divide data into train and test with 15% test data
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 0.15, shuffle = False)
    
# feature scaling
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
    
# create time series split for CV
splits = 4
tscv = TimeSeriesSplit(n_splits = splits)
    
# create linear regressor 
regressor = LinearRegression()
    
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    regressor.fit(X_train_split, y_train_split)
    
# predict for X_test  
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mse_error = mse(y_test, y_pred) # 1479.61335
mae_error = mae(y_test, y_pred) # 23.1525
    
rmse_gen.append(rmse_error)
mse_gen.append(mse_error)
mae_gen.append(mae_error)
    
# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================
    
y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values
    
# smal adjustment
y_test.replace(0, 0.0001,inplace = True)

# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mse_spike = mse(y_test_spike, y_pred_spike)
mae_spike = mae(y_test_spike, y_pred_spike)
    
rmse_spi.append(rmse_spike)
mse_spi.append(mse_spike)
mae_spi.append(mae_spike)
    
# =============================================================================
# Metric evaluation on normal regions
# =============================================================================
    
# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)
    
# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mse_normal = mse(y_test_normal, y_pred_normal)
mae_normal = mae(y_test_normal, y_pred_normal)
    
rmse_nor.append(rmse_normal)
mse_nor.append(mse_normal)
mae_nor.append(mae_normal)  

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


# Data processing (moving Nested CV):

In [3]:
# data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)
data = data.loc[data.index > 2017010000, :]
    
# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)
    
# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

X.fillna(X.mean(), inplace = True)
y.fillna(y.mean(), inplace = True)
    
X = X.astype('float64')
X = X.round(20)
    
# divide data into train and test with 15% test data
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 0.15, shuffle = False)
    
# feature scaling
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
    
# create time series split for CV
splits = 4
# 2 * 48 * 0.85 * 30.5 = 2489
tscv = TimeSeriesSplit(n_splits = splits, max_train_size = 2489)
    
# create linear regressor 
regressor = LinearRegression()
    
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    regressor.fit(X_train_split, y_train_split)
    
# predict for X_test  
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mse_error = mse(y_test, y_pred) # 1479.61335
mae_error = mae(y_test, y_pred) # 23.1525
    
rmse_gen.append(rmse_error)
mse_gen.append(mse_error)
mae_gen.append(mae_error)
    
# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================
    
y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values
    
# smal adjustment
y_test.replace(0, 0.0001,inplace = True)

# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mse_spike = mse(y_test_spike, y_pred_spike)
mae_spike = mae(y_test_spike, y_pred_spike)
    
rmse_spi.append(rmse_spike)
mse_spi.append(mse_spike)
mae_spi.append(mae_spike)
    
# =============================================================================
# Metric evaluation on normal regions
# =============================================================================
    
# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)
    
# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mse_normal = mse(y_test_normal, y_pred_normal)
mae_normal = mae(y_test_normal, y_pred_normal)
    
rmse_nor.append(rmse_normal)
mse_nor.append(mse_normal)
mae_nor.append(mae_normal)  

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [4]:
results = pd.DataFrame({                       
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0,30.829045,22.178709,63.543469,51.004364,20.844857,17.361690
1,426.678705,39.057183,544.102755,94.714823,397.613160,27.359341


In [5]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0,30.829045,22.178709,63.543469,51.004364,20.844857,17.361690
1,426.678705,39.057183,544.102755,94.714823,397.613160,27.359341
